In [3]:
from keras import applications

from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback


from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import (Dropout, Flatten, Dense, Conv2D, 
                          Activation, MaxPooling2D)

from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

from sklearn.cross_validation import train_test_split

import os, glob
from tqdm import tqdm
from collections import Counter
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import shutil

In [12]:
more_im = glob.glob("collie_lab/*/*.jpg")
more_im = shuffle(more_im)
collie = [x for x in more_im if "coll" in x.split("\\")[-2]]
lab = [x for x in shuffle(more_im) if "lab" in x.split("\\")[-2]]

print(len(collie))
print(len(lab))

for_labeling = collie + lab
for_labeling = shuffle(for_labeling)

Counter([x.split("\\")[-2] for x in more_im]).most_common()

600
596


[('collie', 600), ('labrador', 596)]

In [18]:
import shutil
from tqdm import tqdm

%mkdir collie_lab_train
%mkdir collie_lab_valid

%mkdir collie_lab_train\\collie
%mkdir collie_lab_train\\lab

%mkdir collie_lab_valid\\collie
%mkdir collie_lab_valid\\lab


for index, image in tqdm(enumerate(for_labeling)):
    
    if index < 1000:
        label = image.split("\\")[-2]
        image_name = image.split("\\")[-1]
        if "coll" in label:
            shutil.copy(image, 'collie_lab_train\\collie\\{}'.format(image_name))
        if "lab" in label:
            shutil.copy(image, 'collie_lab_train\\lab\\{}'.format(image_name))
            
    if index > 1000:
        label = image.split("\\")[-2]
        image_name = image.split("\\")[-1]
        if "coll" in label:
            shutil.copy(image, 'collie_lab_valid\\collie\\{}'.format(image_name))
        if "lab" in label:
            shutil.copy(image, 'collie_lab_valid\\lab\\{}'.format(image_name))   

A subdirectory or file collie_lab_train already exists.
A subdirectory or file collie_lab_valid already exists.
1196it [00:04, 244.83it/s]


In [20]:

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    'collie_lab_train/',
    target_size=(150, 150),
    batch_size=32,
    shuffle=True,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'collie_lab_valid/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 1000 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [22]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(96))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # binary
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.fit_generator(
    train_generator,
    steps_per_epoch= 2500 // 32, # give me more data
    epochs=30,
    callbacks=[TQDMNotebookCallback()],
    verbose=0,
    validation_data=validation_generator,
    validation_steps= 300 // 16)